# Synthetic Experiments with Multiple Sources in Each Batch with Different Source Sizes

In [1]:
output_path = './outputs/graphs/'
results_path = './outputs/synthetic_results_batch_multiple_sources_varied_sizes/'
baseline_path = './outputs/synthetic_results_batch_multiple_sources/'

In [2]:
import json
import os
import pandas as pd
import numpy as np

In [3]:
results_list = []
for file in [
    f for f in os.listdir(results_path) if f.endswith("json")
]:
    with open(results_path + file) as f:
        results_temp = json.load(f)
    results_list.append(results_temp)

In [4]:
results = {}

In [5]:
for results_dict in results_list:

    for dataset in results_dict:
        if dataset not in results:
            results[dataset] = {}
        
        for corruption_type in results_dict[dataset]:
            if corruption_type not in results[dataset]:
                results[dataset][corruption_type] = {}

            for run in results_dict[dataset][corruption_type]:
                if run not in results[dataset][corruption_type]:
                    results[dataset][corruption_type][run] = {}

                for depression in results_dict[dataset][corruption_type][run]:
                    if depression not in results[dataset][corruption_type][run]:
                        results[dataset][corruption_type][run][depression] = {}
                    
                    for n_sources in results_dict[dataset][corruption_type][run][depression]:
                        results[dataset][corruption_type][run][depression][n_sources] = [
                            dict(epoch=int(epoch), **metrics) 
                            for epoch, metrics in results_dict[dataset][corruption_type][run][depression][n_sources].items() 
                            if epoch != 'corrupt_sources'
                        ]

In [6]:
results_df = []

for dataset in results:
    for corruption_type in results[dataset]:
        for run in results[dataset][corruption_type]:
            for depression in results[dataset][corruption_type][run]:
                for n_sources in results[dataset][corruption_type][run][depression]:
                    results_df.append(
                        pd.json_normalize(
                            results[dataset][corruption_type][run][depression][n_sources]
                        )
                        .assign(
                            dataset=dataset,
                            corruption_type=corruption_type,
                            run=run,
                            depression=depression,
                            n_sources=n_sources
                        )
                        .assign(epoch=lambda x: x['epoch']+1)
                    )

results_df = pd.concat(results_df)
results_df = (
    results_df
    .replace({"depression": {"true": True, "false": False}})
    .astype(
        {
            "run": "int64",
            "n_sources": "int64",
            "depression": "bool",
        }
    )
)

In [7]:
results_df = results_df.loc[lambda df: df["n_sources"] != 10]

In [8]:
with open(os.path.join(baseline_path, "baseline", "coteaching", "results.json")) as f:
    results_cot = json.load(f)


with open(os.path.join(baseline_path, "baseline", "idpa", "results.json")) as f:
    results_idpa = json.load(f)


with open(os.path.join(baseline_path, "results.json")) as f:
    results_lap_10 = json.load(f)

In [9]:
results_cot_df = []

for dataset in results_cot:
    for corruption_type in results_cot[dataset]:
        for run in results_cot[dataset][corruption_type]:
            results_cot_df.append(
                pd.DataFrame(
                    results_cot[dataset][corruption_type][run]
                )
                .T
                .reset_index()
                .rename(columns={"index": "epoch"})
                .assign(
                    dataset=dataset,
                    corruption_type=corruption_type,
                    run=run,
                )
                .assign(epoch=lambda x: x['epoch'].astype(int)+1)
            )

results_cot_df = pd.concat(results_cot_df)
results_cot_df = (
    results_cot_df
    .astype(
        {
            "run": "int64",
        }
    )
)

In [10]:
results_idpa_df = []

for dataset in results_idpa:
    for corruption_type in results_idpa[dataset]:
        for run in results_idpa[dataset][corruption_type]:
            results_idpa_df.append(
                pd.DataFrame(
                    results_idpa[dataset][corruption_type][run]
                )
                .T
                .reset_index()
                .rename(columns={"index": "epoch"})
                .assign(
                    dataset=dataset,
                    corruption_type=corruption_type,
                    run=run,
                )
                .assign(epoch=lambda x: x['epoch'].astype(int)+1)
            )

results_idpa_df = pd.concat(results_idpa_df)
results_idpa_df = (
    results_idpa_df
    .astype(
        {
            "run": "int64",
        }
    )
)

In [11]:
results_lap_10_df = []
for dataset in results_lap_10:
    for corruption_type in results_lap_10[dataset]:
        for run in results_lap_10[dataset][corruption_type]:
            for depression in results_lap_10[dataset][corruption_type][run]:
                results_lap_10_df.append(
                    pd.DataFrame(
                        [
                            dict(epoch=int(epoch), **metrics) 
                            for epoch, metrics in results_lap_10[dataset][corruption_type][run][depression].items() 
                            if epoch != 'corrupt_sources'
                        ]
                    )
                    .assign(
                        dataset=dataset,
                        corruption_type=corruption_type,
                        run=run,
                        depression=depression,
                    )
                    .assign(epoch=lambda x: x['epoch'].astype(int)+1)
                )

results_lap_10_df = pd.concat(results_lap_10_df)
results_lap_10_df = (
    results_lap_10_df
    .replace({"depression": {"true": True, "false": False}})
    .astype(
        {
            "run": "int64",
            "depression": "bool",
        }
    )
)

In [12]:
results_cot_df = results_cot_df.assign(method="Co-teaching", n_sources=-1)
results_idpa_df = results_idpa_df.assign(method="IDPA", n_sources=-1)
results_lap_10_df =  (
    results_lap_10_df
    .assign(
        method=lambda df: df['depression'].map({True: "LAP", False: "Standard"})
    )
    .assign(n_sources=lambda df: np.where(df['method'] == "LAP", 10, -1))

)
results_df = results_df.assign(
    method=lambda df: df['depression'].map({True: "LAP", False: "Standard"})
)

In [13]:
columns_intersection = [
    'dataset', 'corruption_type', 'run', 'epoch', 
    'method', 'test_top1acc', 'test_top5acc', 'n_sources'
]

results_all = (
    pd.concat([
        results_df[columns_intersection],
        results_lap_10_df[columns_intersection],
        results_cot_df[columns_intersection],
        results_idpa_df[columns_intersection],
    ])
    .assign(test_top1acc=lambda x: x['test_top1acc']*100)
    .groupby(['dataset', 'corruption_type', 'run', 'method', 'n_sources'])
    .agg({'test_top1acc': 'max', 'test_top5acc': 'max'})
    .unstack()
    .swaplevel(axis=1)   
    .reset_index()
    .drop(columns='run')
    .groupby(['dataset', 'corruption_type', 'method',])
    .agg(['mean', 'std'])
    .stack(0)
    .stack(0)
    .assign(
        mean_std = lambda x: 
            np.round(x['mean'], 2).astype(str) 
            + " ± "
            + np.round(x['std'], 2).astype(str),
    )
    .drop(columns=['mean', 'std'])
    .unstack()
    .unstack()
    .unstack()
    .dropna(axis=1, how='all')
)

/var/folders/dn/w2011ntd5gxf26dgf_38535m0000gp/T/ipykernel_12764/622575766.py:19: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  .drop(columns='run')
/var/folders/dn/w2011ntd5gxf26dgf_38535m0000gp/T/ipykernel_12764/622575766.py:21: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  .agg(['mean', 'std'])


In [14]:
def bold_max_value_latex(x, model_names):
    x = x.copy()
    len_cols = x.shape[0]
    n_models = len(model_names)
    idx_bold = (
        x
        [-n_models:]
        .str.replace(" ", "")
        .str.split("±")
        .str[0]
        .argmax()
    )
    x.iloc[idx_bold+len_cols-n_models] = '\\textbf{' + x.iloc[idx_bold+len_cols-n_models] + '}'
    
    return x

def bold_max_value_html(x, model_names):
    x = x.copy()
    len_cols = x.shape[0]
    n_models = len(model_names)
    idx_bold = (
        x
        [-n_models:]
        .str.replace(" ", "")
        .str.split("±")
        .str[0]
        .argmax()
    )
    x.iloc[idx_bold+len_cols-n_models] = '<strong>' + x.iloc[idx_bold+len_cols-n_models] + '</strong>'
    
    return x

def bold_max_value_md(x, model_names):
    x = x.copy()
    len_cols = x.shape[0]
    n_models = len(model_names)
    idx_bold = (
        x
        [-n_models:]
        .str.replace(" ", "")
        .str.split("±")
        .str[0]
        .argmax()
    )
    x.iloc[idx_bold+len_cols-n_models] = '**' + x.iloc[idx_bold+len_cols-n_models] + '**'
    
    return x

corruption_type_order = [
    'Original Data',
    'Chunk Shuffle',
    'Random Label',
    'Batch Label Shuffle',
    'Batch Label Flip',
    'Added Noise',
    'Replace With Noise',
]

corruption_type_map = {
    "no_c": "Original Data",
    "c_cs": "Chunk Shuffle",
    "c_rl": "Random Label",
    "c_lbs": "Batch Label Shuffle",
    "c_lbf": "Batch Label Flip",
    "c_ns": "Added Noise",
    "c_no": "Replace With Noise",
}

model_order = [
    'Standard',
    'IDPA',
    'Co-teaching',
    'LAP-10',
    'LAP-50',
    'LAP-250',
    'LAP-1250',
    'LAP-6250',
    'LAP-50000',
]


def flatten_columns(df):
    df.columns = [''.join(col if isinstance(col, str) else str(col)).strip() for col in df.columns.values]
    return df



results_formatted = (
    results_all
    .loc[:, ('mean_std', 'test_top1acc')]
    .pipe(flatten_columns)
    .rename(
        columns={
            "(-1, 'Standard')": 'Standard',
            "(-1, 'Co-teaching')": 'Co-teaching',
            "(-1, 'IDPA')": 'IDPA',
            "(10, 'LAP')": 'LAP-10',
            "(50, 'LAP')": 'LAP-50',
            "(250, 'LAP')": 'LAP-250',
            "(1250, 'LAP')": 'LAP-1250',
            "(6250, 'LAP')": 'LAP-6250',
            "(50000, 'LAP')": 'LAP-50000',
        }
    )
    # makes bold with latex:
    .apply(
        bold_max_value_latex,
        model_names = model_order,
        axis=1
    )
    .loc['cifar10']
    .rename_axis(index=None, columns=None)
    .reset_index()
    .rename(columns={"index": "Corruption Type"})
    .assign(
        **{
            "Corruption Type": lambda x: x['Corruption Type'].map(corruption_type_map),
        }
    )
    .set_index('Corruption Type')
    .loc[corruption_type_order, model_order]    
    .reset_index()
)
results_formatted

,Corruption Type,Standard,IDPA,Co-teaching,LAP-10,LAP-50,LAP-250,LAP-1250,LAP-6250,LAP-50000
0,Original Data,69.46 ± 1.12,\textbf{70.86 ± 0.32},67.73 ± 0.83,69.6 ± 1.08,70.06 ± 0.84,69.73 ± 1.0,69.57 ± 1.07,70.03 ± 0.84,70.28 ± 0.5
1,Chunk Shuffle,65.92 ± 1.32,66.34 ± 0.99,63.55 ± 0.41,66.92 ± 0.56,\textbf{67.3 ± 0.77},65.35 ± 0.98,66.08 ± 1.02,65.45 ± 0.44,65.47 ± 0.98
2,Random Label,60.85 ± 1.67,57.93 ± 1.59,61.37 ± 1.01,66.5 ± 0.43,\textbf{66.76 ± 0.42},66.33 ± 0.56,66.15 ± 1.16,66.42 ± 0.34,66.49 ± 0.65
3,Batch Label Shuffle,62.23 ± 0.62,60.08 ± 0.13,63.9 ± 0.44,66.09 ± 0.65,66.22 ± 0.96,65.55 ± 0.91,\textbf{66.71 ± 0.48},66.43 ± 0.92,65.73 ± 1.13
4,Batch Label Flip,59.94 ± 2.48,61.02 ± 1.9,63.05 ± 2.43,64.48 ± 1.11,\textbf{67.32 ± 0.64},64.74 ± 3.05,66.42 ± 1.22,66.17 ± 1.45,65.41 ± 1.65
5,Added Noise,59.86 ± 1.38,61.19 ± 0.33,59.59 ± 1.37,\textbf{63.58 ± 1.03},60.39 ± 0.55,59.62 ± 0.76,59.52 ± 1.06,59.75 ± 1.08,60.55 ± 1.08
6,Replace With Noise,65.07 ± 0.63,65.85 ± 0.64,64.1 ± 1.13,64.89 ± 1.28,65.45 ± 0.56,65.46 ± 0.92,\textbf{66.39 ± 1.66},64.94 ± 0.44,65.09 ± 0.43


In [15]:
model_order = [
    'Standard', 'IDPA', 'Co-teaching', 'LAP-10',
    'LAP-1250', 'LAP-50000'
]

print(
    results_all
    .loc[:, ('mean_std', 'test_top1acc')]
    .pipe(flatten_columns)
    .rename(
        columns={
            "(-1, 'Standard')": 'Standard',
            "(-1, 'Co-teaching')": 'Co-teaching',
            "(-1, 'IDPA')": 'IDPA',
            "(10, 'LAP')": 'LAP-10',
            "(50, 'LAP')": 'LAP-50',
            "(250, 'LAP')": 'LAP-250',
            "(1250, 'LAP')": 'LAP-1250',
            "(6250, 'LAP')": 'LAP-6250',
            "(50000, 'LAP')": 'LAP-50000',
        }
    )
    .loc[:, model_order]
    # makes bold with latex:
    .apply(
        bold_max_value_latex,
        model_names = model_order,
        axis=1
    )
    .loc['cifar10']
    .rename_axis(index=None, columns=None)
    .reset_index()
    .rename(columns={"index": "Corruption Type"})
    .assign(
        **{
            "Corruption Type": lambda x: x['Corruption Type'].map(corruption_type_map),
        }
    )
    .set_index('Corruption Type')
    .loc[corruption_type_order, model_order]    
    .reset_index()
    .to_latex(index=False)
)

\begin{tabular}{lllllll}
\toprule
Corruption Type & Standard & IDPA & Co-teaching & LAP-10 & LAP-1250 & LAP-50000 \\
\midrule
Original Data & 69.46 ± 1.12 & \textbf{70.86 ± 0.32} & 67.73 ± 0.83 & 69.6 ± 1.08 & 69.57 ± 1.07 & 70.28 ± 0.5 \\
Chunk Shuffle & 65.92 ± 1.32 & 66.34 ± 0.99 & 63.55 ± 0.41 & \textbf{66.92 ± 0.56} & 66.08 ± 1.02 & 65.47 ± 0.98 \\
Random Label & 60.85 ± 1.67 & 57.93 ± 1.59 & 61.37 ± 1.01 & \textbf{66.5 ± 0.43} & 66.15 ± 1.16 & 66.49 ± 0.65 \\
Batch Label Shuffle & 62.23 ± 0.62 & 60.08 ± 0.13 & 63.9 ± 0.44 & 66.09 ± 0.65 & \textbf{66.71 ± 0.48} & 65.73 ± 1.13 \\
Batch Label Flip & 59.94 ± 2.48 & 61.02 ± 1.9 & 63.05 ± 2.43 & 64.48 ± 1.11 & \textbf{66.42 ± 1.22} & 65.41 ± 1.65 \\
Added Noise & 59.86 ± 1.38 & 61.19 ± 0.33 & 59.59 ± 1.37 & \textbf{63.58 ± 1.03} & 59.52 ± 1.06 & 60.55 ± 1.08 \\
Replace With Noise & 65.07 ± 0.63 & 65.85 ± 0.64 & 64.1 ± 1.13 & 64.89 ± 1.28 & \textbf{66.39 ± 1.66} & 65.09 ± 0.43 \\
\bottomrule
\end{tabular}

